In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160704)
tf.set_random_seed(20160704)

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
num_filters1 = 32

x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1])

W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,num_filters1],
                                                                             stddev=0.1))
h_conv1 = tf.nn.conv2d(x_image, W_conv1,
                                            strides=[1,1,1,1], padding='SAME')
b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]))

h_conv1_cutoff = tf.nn.relu(h_conv1 + b_conv1)

h_pool1 = tf.nn.max_pool(h_conv1_cutoff, ksize=[1,2,2,1],
                                              strides=[1,2,2,1], padding='SAME')

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
num_filters2 = 64

W_conv2 = tf.Variable(
                        tf.truncated_normal([5,5,num_filters1,num_filters2],
                                                              stddev=0.1))
h_conv2 = tf.nn.conv2d(h_pool1, W_conv2,
                                            strides=[1,1,1,1], padding='SAME')

b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]))

h_conv2_cutoff = tf.nn.relu(h_conv2 + b_conv2)

h_pool2 = tf.nn.max_pool(h_conv2_cutoff, ksize=[1,2,2,1],
                                              strides=[1,2,2,1], padding='SAME')

In [5]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*num_filters2])

num_units1 = 7*7*num_filters2
num_units2 = 1024

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b2 = tf.Variable(tf.constant(0.1, shape=[num_units2]))
hidden2 = tf.nn.relu(tf.matmul(h_pool2_flat, w2) + b2)

keep_prob = tf.placeholder(tf.float32)
hidden2_drop = tf.nn.dropout(hidden2, keep_prob)

w0 = tf.Variable(tf.zeros([num_units2, 10]))
b0 = tf.Variable(tf.zeros([10]))
p = tf.nn.softmax(tf.matmul(hidden2_drop, w0) + b0)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
t = tf.placeholder(tf.float32, [None, 10])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [7]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())
saver = tf.train.Saver()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [11]:
i = 0
for _ in range(20000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(50)
    sess.run(train_step,
             feed_dict={x:batch_xs, t:batch_ts, keep_prob:0.5})
    if i % 500 == 0:
        loss_vals, acc_vals = [], []
        for c in range(4):
            start = len(mnist.test.labels) / 4 * c
            end = len(mnist.test.labels) / 4 * (c+1)
            loss_val, acc_val = sess.run([loss, accuracy],
                feed_dict={x:mnist.test.images[int(start):int(end)],
                           t:mnist.test.labels[int(start):int(end)],
                           keep_prob:1.0})
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
        loss_val = np.sum(loss_vals)
        acc_val = np.mean(acc_vals)
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))
        saver.save(sess, 'cnn_session', global_step=i)

Step: 500, Loss: 678.237366, Accuracy: 0.976500
Step: 1000, Loss: 598.362122, Accuracy: 0.980800
Step: 1500, Loss: 489.249390, Accuracy: 0.983800
Step: 2000, Loss: 475.128906, Accuracy: 0.984100
Step: 2500, Loss: 450.201416, Accuracy: 0.984800
Step: 3000, Loss: 409.029053, Accuracy: 0.987000
Instructions for updating:
Use standard file APIs to delete files with this prefix.
Step: 3500, Loss: 385.544373, Accuracy: 0.987100
Step: 4000, Loss: 381.282074, Accuracy: 0.986800
Step: 4500, Loss: 383.194702, Accuracy: 0.986000
Step: 5000, Loss: 323.370178, Accuracy: 0.989200
Step: 5500, Loss: 360.556580, Accuracy: 0.986300
Step: 6000, Loss: 330.961792, Accuracy: 0.988100
Step: 6500, Loss: 365.201050, Accuracy: 0.987300
Step: 7000, Loss: 293.353485, Accuracy: 0.989500
Step: 7500, Loss: 285.675171, Accuracy: 0.990000
Step: 8000, Loss: 297.789337, Accuracy: 0.990100
Step: 8500, Loss: 324.375244, Accuracy: 0.989000
Step: 9000, Loss: 293.919617, Accuracy: 0.989900
Step: 9500, Loss: 276.651489, Accur